# Data Analytics Project
## Heart Failure Classification
Dataset provided by [Kaggle](https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction)
### 1. Data Preparation

In [ ]:
# data wrangling and visualisation libraries
import pandas as pd 
import numpy as np 
import seaborn as sns 
from matplotlib import pyplot as plt 
import sklearn 

# prevents warnings from happening
import warnings 
warnings.simplefilter('ignore', category=UserWarning)

# makes plots pretty in jupyternotebook
%matplotlib inline

df = pd.read_csv()